## GoNB 的基本使用

**下面是一个最简单的 go demo 程序：**

In [1]:
package main

import "fmt"

func main() {
    fmt.Println("Hello, world!")
}

Hello, world!


但是在 GoNB 中可以不这么麻烦。

**首先，gonb 会自动尝试 import 依赖包**,(builtin 类型的包，第三方包需要手动 import)

**其次，在一个 code cell 不需要每次都写一遍`func main() {}`，使用 `%%`可以简化这个过程，在一个 cell 里面加上 `%%`, gonb 会自动将代码插入 `func main() {}` 内部。**

In [2]:
%%
fmt.Println("hello, gonb")

hello, gonb


### 1. 变量

***import、函数、常量、类型和变量全局声明在执行后会被记住，并从一个 cell 传到另一个 cell。***

**但是对于变量的声明，其执行结果不能在 cell 间共享(传递)**

In [3]:
// 在一个 cell 中定义一个函数
func incr[T interface{constraints.Float|constraints.Integer}](x T) T {
    return x+T(1)
}

In [4]:
// 另一个 cell 中使用
%%
x := incr(1)
y := incr(math.Pi)
fmt.Printf("incr: x=%d, y=%f\n", x, y)

incr: x=2, y=4.141593


In [5]:
// 声明一个变量
var startValue = float32(1)

%%
// 更新这个值：
startValue = incr(startValue)
fmt.Printf("current startValue=%f\n", startValue)

current startValue=2.000000


In [6]:
// 但是在另外一个 cell 中，其值并没有传递过来，还是定义时的值：
%%
fmt.Printf("current startValue=%f\n", startValue)

current startValue=1.000000


**要在执行结果保存到下一个 cell，可以用 `github.com/janpfeifer/gonb/cache` 包**

多次执行下面这段代码，`CachedValue`值一直不变，而 `NonCachedValue` 一直变化：

In [7]:
// Temporary fix until new release v0.6.0 propagates.
import (
    "math/rand"
    "github.com/janpfeifer/gonb/cache"
)

func VeryExpensive() int {
    fmt.Println("\t...VeryExpensive() call...")
    return rand.Intn(1000)
}

var (
    CachedValue = cache.Cache("expensive", VeryExpensive)
    NonCachedValue = VeryExpensive()
)
    
%%
fmt.Printf("NonCachedValue=%d\n", NonCachedValue)
fmt.Printf("   CachedValue=%d\n", CachedValue)

	...VeryExpensive() call...
NonCachedValue=828
   CachedValue=86


移除定义的变量，免得对后面的代码有影响

In [8]:
%rm NonCachedValue CachedValue

. removed var NonCachedValue
. removed var CachedValue


### 2. import

1. gonb 会在编译代码前 执行 **goimports**， 这会自动添加代码里涉及到的 builtin packages
2. gonb 会在编译代码前执行 **go get**， 来自动下载在代码里 import 的第三方包
3. 当然我们还可以手动下载，这个后面会用特殊的命令实现

In [9]:
// 下面是一个例子，第三方包 progressbar 是第一次使用，但是在执行这段代码时， gonb 会自动获取
import progressbar "github.com/schollz/progressbar/v3"

%%
bar := progressbar.NewOptions(100, 
                              progressbar.OptionUseANSICodes(true),
                              progressbar.OptionShowIts(),
                              progressbar.OptionSetItsString("steps"))
for i := 0; i < 100; i++ {
    bar.Add(1)
    time.Sleep(40 * time.Millisecond)
}
fmt.Printf("\nDone\n")

 100% |████████████████████████████████████████| (25 steps/s) [3s:0s]0s]
Done


### 3. 显示 HTML，Markdown，image 等到

**DisplayHtml**

In [10]:
// DisplayHtml
import "github.com/janpfeifer/gonb/gonbui"

%%
gonbui.DisplayHtml(`<span style="background:pink; color:#111; border-radius: 3px; border: 3px solid orange; font-size: 18px;">I 🧡 GoNB!</span>`)

I 🧡 GoNB!

**Markdown**

In [11]:
// DisplayMarkdown
%%
gonbui.DisplayMarkdown("#### Objective\n\n1. Have fun coding **Go**;\n1. Profit...\n"+
                       `$$f(x) = \int_{-\infty}^{\infty} e^{-x^2} dx$$`)

#### Objective

1. Have fun coding **Go**;
1. Profit...
$$f(x) = \int_{-\infty}^{\infty} e^{-x^2} dx$$

**绘图**

首先手动安装外部包：

In [12]:
!*go get -u github.com/erkkah/margaid@d60b2efd2f5acc5d8fbbe13eaf85f1532e11a2fb

go: added github.com/erkkah/margaid v0.1.1-0.20230128143048-d60b2efd2f5a


In [13]:
import "bytes"
import "github.com/janpfeifer/gonb/gonbui"
import mg "github.com/erkkah/margaid"

func mgPlot(width, height int) string {
    randomSeries := mg.NewSeries()
    rand.Seed(time.Now().Unix())
    for i := float64(0); i < 10; i++ {
        randomSeries.Add(mg.MakeValue(i+1, 200*rand.Float64()))
    }

    testSeries := mg.NewSeries()
    multiplier := 2.1
    v := 0.33
    for i := float64(0); i < 10; i++ {
        v *= multiplier
        testSeries.Add(mg.MakeValue(i+1, v))
    }

    diagram := mg.New(width, height,
        mg.WithAutorange(mg.XAxis, testSeries),
        mg.WithAutorange(mg.YAxis, testSeries),
        mg.WithAutorange(mg.Y2Axis, testSeries),
        mg.WithProjection(mg.YAxis, mg.Log),
        mg.WithInset(70),
        mg.WithPadding(2),
        mg.WithColorScheme(90),
        mg.WithBackgroundColor("#f8f8f8"),
    )

    diagram.Line(testSeries, mg.UsingAxes(mg.XAxis, mg.YAxis), mg.UsingMarker("square"), mg.UsingStrokeWidth(1))
    diagram.Smooth(testSeries, mg.UsingAxes(mg.XAxis, mg.Y2Axis), mg.UsingStrokeWidth(3.14))
    diagram.Smooth(randomSeries, mg.UsingAxes(mg.XAxis, mg.YAxis), mg.UsingMarker("filled-circle"))
    diagram.Axis(testSeries, mg.XAxis, diagram.ValueTicker('f', 0, 10), false, "X")
    diagram.Axis(testSeries, mg.YAxis, diagram.ValueTicker('f', 1, 2), true, "Y")

    diagram.Frame()
    diagram.Title("A diagram of sorts 📊 📈")
    buf := bytes.NewBuffer(nil)
    diagram.Render(buf)
    return buf.String()
}

%%
gonbui.DisplaySvg(mgPlot(640, 480))

<path vector-effect="non-scaling-stroke" d="M1.000000e+01,6.800000e+00 C1.888889e+01,6.875434e+00 4.555556e+01,7.018758e+00 6.333333e+01,7.252602e+00 C8.111111e+01,7.486446e+00 9.888889e+01,7.711993e+00 1.166667e+02,8.203066e+00 C1.344444e+02,8.694139e+00 1.522222e+02,9.167786e+00 1.700000e+02,1.019904e+01 C1.877778e+02,1.123029e+01 2.055556e+02,1.222495e+01 2.233333e+02,1.439059e+01 C2.411111e+02,1.655622e+01 2.588889e+02,1.864500e+01 2.766667e+02,2.319283e+01 C2.944444e+02,2.774066e+01 3.122222e+02,3.212711e+01 3.300000e+02,4.167755e+01 C3.477778e+02,5.122798e+01 3.655556e+02,6.043953e+01 3.833333e+02,8.049545e+01 C4.011111e+02,1.005514e+02 4.188889e+02,1.198956e+02 4.366667e+02,1.620130e+02 C4.544444e+02,2.041305e+02 4.811111e+02,3.046688e+02 4.900000e+02,3.332000e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.000000e+01,2.354585e+02 C1.888889e+01,2.239171e+02 4.555556e+01,1.608913e+02 6.333333e+01,1.662100e+02 C8.111111e+01,1.715288e+02 9.888889e+01,2.521869e+02 1.166667e+02,2.673708e+02 C1.344444e+02,2.825548e+02 1.522222e+02,2.603916e+02 1.700000e+02,2.573139e+02 C1.877778e+02,2.542362e+02 2.055556e+02,2.552644e+02 2.233333e+02,2.489046e+02 C2.411111e+02,2.425449e+02 2.588889e+02,2.136328e+02 2.766667e+02,2.191553e+02 C2.944444e+02,2.246778e+02 3.122222e+02,2.728981e+02 3.300000e+02,2.820395e+02 C3.477778e+02,2.911810e+02 3.655556e+02,2.738637e+02 3.833333e+02,2.740040e+02 C4.011111e+02,2.741443e+02 4.188889e+02,2.838430e+02 4.366667e+02,2.828814e+02 C4.544444e+02,2.819197e+02 4.811111e+02,2.706752e+02 4.900000e+02,2.682339e+02 "/> 1 2 3 4 5 6 7 8 9 10 X 1.0 2.0 4.0 8.0 16.0 32.0 64.0 128.0 256.0 512.0 Y A diagram of sorts 📊 📈

### 4. init() 函数

由于每个 cell 可以共用变量，包括函数定义，所以每个 cell 不能有自己的 init() 函数，这会导致重复定义。

在 gonb 中，是这样处理的，每个 cell 可以有 `func init_something()`, 然后在编译时 gonb 会自动将它们转换为 `func init()`, 这样就能在自己的 cell 中执行自己的 init() 函数了：

In [14]:
// cell a
func init_a() {
    fmt.Println("init_a")
}
%%
fmt.Println("main")

init_a
main


In [15]:
// 先执行了 cell a 中的 init() 函数，然后执行本 cell 的 init() 函数，最后是 main() 函数
func init_b() {
    fmt.Println("init_b")
}
%%
fmt.Println("main")

init_a
init_b
main


如果不需要，可以移除，免得对后面的代码产生影响：

In [16]:
%rm init_a init_b

. removed func init_a
. removed func init_b


### 5. Flags

**%%** 命令不仅会把它后面的代码 warp 到 `func main() {}`里面，它还能设置参数

In [17]:
// 首先定义个名为 who 的 flag 变量，默认为空
var flagWho = flag.String("who", "", "Your name!")

// 给 who 这个变量赋值
%% --who=world
fmt.Printf("Hello %s!\n", *flagWho)

Hello world!


同样可以使用 **%args** 来对参数进行赋值

In [18]:
// 给 args 类型变量 who 赋值
%args --who=Wally

func main() {
    // 解析 args
    flag.Parse()
    fmt.Printf("Where is %s?", *flagWho)
}


Where is Wally?

### 6. 执行 shell 命令

有两种方式执行 shell 命令：
 1. `!`前缀 + shell 命令
 2. `!*`前缀 + shell 命令

它们唯一不同就是它们执行命令的目录不同。

前者是当前 `.ipynb` 文件所在目录，后者是当前 go 程序执行编译代码的项目的目录，也就是包含 `main.go`, `go.mod` 等文件的目录。
我们手动安装 go 依赖包时就是用的这个命令。

In [19]:
!go version

go version go1.23.0 linux/amd64


In [20]:
!pwd && ls -l

/notebooks
total 244
-rw-r--r--  1 jovyan users  17801 Sep  9 04:12 demo.ipynb
drwxr-xr-x 12 jovyan users   4096 Aug 18 07:27 gonb
-rw-r--r--  1 root   root  222917 Jul 10 07:19 tutorial.ipynb


In [21]:
!* pwd && ls -l

/tmp/gonb_e03b1eee
total 2248
-rw-r--r-- 1 jovyan users     633 Sep  9 04:13 go.mod
-rwxr-xr-x 1 jovyan users 2288838 Sep  9 04:13 gonb_e03b1eee
srwxr-xr-x 1 jovyan users       0 Sep  9 04:12 gopls_socket
-rw-r--r-- 1 jovyan users    2983 Sep  9 04:13 go.sum
-rw-r--r-- 1 jovyan users    1898 Sep  9 04:13 main.go


执行一般 shell 命令就用 `!` 前缀了：

In [22]:
!((ii=0)) ;\
while ((ii < 5)) ; do \
  printf "\rCounting: ${ii} ..." ;\
  sleep 1;\
  ((ii+=1));\
done;\
echo

Counting: 4 ...


### 7. 测试以及 Benchmarks

执行 `go test` 在这里可以用 `%test` 代替：

In [23]:
import "github.com/stretchr/testify/require"

func TestIncr(t *testing.T) {
    require.Equal(t, 2, incr(1))    
}

func BenchmarkIncr(b *testing.B) {
    for i := 0; i < b.N; i++ {
        _ = incr(i)
    }
}

%test
// 也可以这样：
%test -test.bench=. -test.run=Bechmark

goos: linux
goarch: amd64
pkg: gonb_e03b1eee
cpu: AMD Ryzen 5 4500U with Radeon Graphics         
BenchmarkIncr-6   	1000000000	         0.2689 ns/op
PASS


### 8. 环境变量

有以下环境变量：

- `GONB_DIR`: gonb 中命令执行的目录，可以通过 **%cd** 更改。
- `GONB_TMP_DIR`: 执行 go 代码单元的临时目录，也就是 `!*` 执行的目录。当kernel 重启后，这个目录销毁并新建
- `GONB_PIPE`: 跟 `gonbui `相关，即显示 html, image 等富文本相关。


### 9. 获取帮助

命令 `%help`